<a href="https://colab.research.google.com/github/ParthDave111/Quant-and-Finance-File/blob/main/European_Option.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def european_option_trinomial(S0, K, T, r, sigma, N, option_type='call'):
    # Calculate parameters
    dt = T / N
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = 1 / u
    m = 1

    # Risk-neutral probabilities for a trinomial tree
    pu = ((np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) /
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) /
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2))))**2
    pm = 1 - pu - pd

    discount = np.exp(-r * dt)

    # Initialize asset prices at maturity
    asset_prices = np.zeros((2 * N + 1, N + 1))
    asset_prices[N, 0] = S0

    for i in range(1, N + 1):
        for j in range(N - i, N + i + 1, 2):
            asset_prices[j, i] = S0 * (u ** ((j - N + i) // 2)) * (d ** ((N + i - j) // 2))

    # Initialize option values at maturity
    option_values = np.zeros((2 * N + 1, N + 1))

    if option_type == 'call':
        option_values[:, N] = np.maximum(0, asset_prices[:, N] - K)
    elif option_type == 'put':
        option_values[:, N] = np.maximum(0, K - asset_prices[:, N])

    # Backward induction for European option pricing
    for i in range(N - 1, -1, -1):
        for j in range(N - i, N + i + 1, 2):
            # Calculate the option values using risk-neutral probabilities
            option_values[j, i] = discount * (
                pu * option_values[j - 1, i + 1] +
                pm * option_values[j, i + 1] +
                pd * option_values[j + 1, i + 1]
            )

    return option_values[N, 0]

# Example usage
S0 = 100  # Initial stock price
K_arr = [60, 90, 100, 110, 140]   # Different strike prices for Deep OTM, OTM, ATM, ITM, and Deep ITM
T = 1     # Time to maturity (in years)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
N = 50    # Number of time steps

for K in K_arr:
    call_price = european_option_trinomial(S0, K, T, r, sigma, N, option_type='call')
    put_price = european_option_trinomial(S0, K, T, r, sigma, N, option_type='put')
    print(f"European Call Option Price for K = {K}: {np.round(call_price, 2)}")
    print(f"European Put Option Price for K = {K}: {np.round(put_price, 2)}")


European Call Option Price for K = 60: 0.0
European Put Option Price for K = 60: 0.0
European Call Option Price for K = 90: 0.0
European Put Option Price for K = 90: 0.0
European Call Option Price for K = 100: 0.0
European Put Option Price for K = 100: 0.0
European Call Option Price for K = 110: 0.0
European Put Option Price for K = 110: 0.0
European Call Option Price for K = 140: 0.0
European Put Option Price for K = 140: 0.0
